In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import joblib
import numpy as np

random_state = 42

In [14]:
def read_dataset(path, sep=';', encoding='utf-8-sig', low_memory=False):
    return pd.read_csv(path, sep=sep, encoding=encoding, low_memory=low_memory)

def save_dataset(dataset, path, sep=';', index=False, encoding='utf-8-sig'):
    dataset.to_csv(path, sep=sep, index=index, encoding=encoding)

course_1_bak_spec = read_dataset("../../data/datasets/1st_course_bak_spec.csv")
course_1_magistr = read_dataset("../../data/datasets/1st_course_magistr.csv")

subjects_bak_spec = read_dataset("../../data/datasets/subjects_bak_spec.csv")
subjects_magistr = read_dataset("../../data/datasets/subjects_magistr.csv")

ОБРАБОТКА ДАТАСЕТОВ ДЛЯ ПЕРВОГО КУРСА

In [16]:
df_numeric = course_1_bak_spec.select_dtypes(include=[np.number])
df_numeric.dropna(inplace=True)
corr_matrix = df_numeric.corr().abs()
np.fill_diagonal(corr_matrix.values, 0)
pairs = (
    corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    .stack()
    .sort_values(ascending=False)
)
top_pairs = pairs.head(20)
for (col1, col2), value in top_pairs.items():
    print(f"{col1} ↔ {col2}: {value:.4f}")

Полных лет на момент поступления ↔ Прошло лет с окончания уч. заведения: 0.8682
Отдельная квота ↔ Боевые действия: 0.8147
БВИ ↔ олимпиада из перечня, утвержденного МОН РФ (ОШ): 0.7281
Особая квота ↔ Инвалиды: 0.7202
Cумма баллов испытаний ↔ БВИ: 0.7156
Особая квота ↔ Сироты: 0.6477
БВИ ↔ Иностранный абитуриент (МОН): 0.6198
Нуждается в общежитии ↔ fromEkaterinburg: 0.5823
fromEkaterinburg ↔ fromSverdlovskRegion: 0.5711
Иностранный абитуриент (МОН) ↔ Квота для иностранных граждан: 0.5571
Полных лет на момент поступления ↔ Высшее: 0.5368
Cумма баллов испытаний ↔ олимпиада из перечня, утвержденного МОН РФ (ОШ): 0.5211
Полных лет на момент поступления ↔ Заочная: 0.4839
Заочная ↔ СПО: 0.4816
Специалист ↔ Код направления 1: 10: 0.4748
Прошло лет с окончания уч. заведения ↔ Высшее: 0.4702
Полных лет на момент поступления ↔ СПО: 0.4557
Cумма баллов испытаний ↔ Иностранный абитуриент (МОН): 0.4435
Код направления 3: 3 ↔ Код направления 3: 4: 0.3883
Нуждается в общежитии ↔ fromSverdlovskRegion: 

In [18]:
course_1_bak_spec.drop(columns=['Прошло лет с окончания уч. заведения'], inplace=True)

In [17]:
df_numeric = course_1_magistr.select_dtypes(include=[np.number])
df_numeric.dropna(inplace=True)
corr_matrix = df_numeric.corr().abs()
np.fill_diagonal(corr_matrix.values, 0)
pairs = (
    corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    .stack()
    .sort_values(ascending=False)
)
top_pairs = pairs.head(20)
for (col1, col2), value in top_pairs.items():
    print(f"{col1} ↔ {col2}: {value:.4f}")

БВИ ↔ Иностранный абитуриент (МОН): 1.0000
Полных лет на момент поступления ↔ Прошло лет с окончания уч. заведения: 0.9440
Иностранный абитуриент (МОН) ↔ Квота для иностранных граждан: 0.7431
БВИ ↔ Квота для иностранных граждан: 0.7431
fromEkaterinburg ↔ fromSverdlovskRegion: 0.6493
Заочная ↔ Код направления 3: 3: 0.5965
Cумма баллов испытаний ↔ Контракт: 0.5250
БВИ ↔ Human Development Index: 0.5016
Иностранный абитуриент (МОН) ↔ Human Development Index: 0.5016
Cумма баллов испытаний ↔ БВИ: 0.4905
Cумма баллов испытаний ↔ Иностранный абитуриент (МОН): 0.4905
Код направления 1: 27 ↔ Код направления 3: 4: 0.4904
Контракт ↔ Полных лет на момент поступления: 0.4207
Контракт ↔ Прошло лет с окончания уч. заведения: 0.4016
Контракт ↔ fromSverdlovskRegion: 0.3809
Cумма баллов испытаний ↔ Квота для иностранных граждан: 0.3645
Human Development Index ↔ Квота для иностранных граждан: 0.3428
Cумма баллов испытаний ↔ Полных лет на момент поступления: 0.3321
Приоритет ↔ Код направления 1: 11: 0.3310

In [19]:
course_1_magistr.drop(columns=['БВИ', 'Прошло лет с окончания уч. заведения'], inplace=True)

In [20]:
save_dataset(course_1_bak_spec, "../../data/datasets/1st_course_bak_spec.csv")
save_dataset(course_1_magistr, "../../data/datasets/1st_course_magistr.csv")

ОБРАБОТКА ДАТАСЕТОВ ДЛЯ БАЛЛОВ

In [21]:
df_numeric = subjects_bak_spec.select_dtypes(include=[np.number])
df_numeric.dropna(inplace=True)
corr_matrix = df_numeric.corr().abs()
np.fill_diagonal(corr_matrix.values, 0)
pairs = (
    corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    .stack()
    .sort_values(ascending=False)
)
top_pairs = pairs.head(20)
for (col1, col2), value in top_pairs.items():
    print(f"{col1} ↔ {col2}: {value:.4f}")

Полных лет на момент поступления ↔ Прошло лет с окончания уч. заведения: 0.8682
Общее количество долгов ↔ Позиция студента в рейтинге: 0.8168
Отдельная квота ↔ Боевые действия: 0.8147
БВИ ↔ олимпиада из перечня, утвержденного МОН РФ (ОШ): 0.7281
Особая квота ↔ Инвалиды: 0.7202
Cумма баллов испытаний ↔ БВИ: 0.7156
Особая квота ↔ Сироты: 0.6477
БВИ ↔ Иностранный абитуриент (МОН): 0.6198
Нуждается в общежитии ↔ fromEkaterinburg: 0.5823
fromEkaterinburg ↔ fromSverdlovskRegion: 0.5711
Иностранный абитуриент (МОН) ↔ Квота для иностранных граждан: 0.5571
Полных лет на момент поступления ↔ Высшее: 0.5368
Общее количество долгов ↔ Код направления 1: 11: 0.5238
Cумма баллов испытаний ↔ олимпиада из перечня, утвержденного МОН РФ (ОШ): 0.5211
Полных лет на момент поступления ↔ Заочная: 0.4839
Таргет ↔ Позиция студента в рейтинге: 0.4826
Заочная ↔ СПО: 0.4816
Специалист ↔ Код направления 1: 10: 0.4748
Прошло лет с окончания уч. заведения ↔ Высшее: 0.4702
Полных лет на момент поступления ↔ СПО: 0.45

In [23]:
subjects_bak_spec.drop(columns=['Прошло лет с окончания уч. заведения'], inplace=True)

In [22]:
df_numeric = subjects_magistr.select_dtypes(include=[np.number])
df_numeric.dropna(inplace=True)
corr_matrix = df_numeric.corr().abs()
np.fill_diagonal(corr_matrix.values, 0)
pairs = (
    corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    .stack()
    .sort_values(ascending=False)
)
top_pairs = pairs.head(20)
for (col1, col2), value in top_pairs.items():
    print(f"{col1} ↔ {col2}: {value:.4f}")

БВИ ↔ Иностранный абитуриент (МОН): 1.0000
Полных лет на момент поступления ↔ Прошло лет с окончания уч. заведения: 0.9440
Общее количество долгов ↔ Позиция студента в рейтинге: 0.8619
Общее количество пересдач ↔ Код направления 1: 27: 0.8544
Иностранный абитуриент (МОН) ↔ Квота для иностранных граждан: 0.7431
БВИ ↔ Квота для иностранных граждан: 0.7431
fromEkaterinburg ↔ fromSverdlovskRegion: 0.6493
Заочная ↔ Код направления 3: 3: 0.5965
Cумма баллов испытаний ↔ Контракт: 0.5250
БВИ ↔ Human Development Index: 0.5016
Иностранный абитуриент (МОН) ↔ Human Development Index: 0.5016
Cумма баллов испытаний ↔ Иностранный абитуриент (МОН): 0.4905
Cумма баллов испытаний ↔ БВИ: 0.4905
Код направления 1: 27 ↔ Код направления 3: 4: 0.4904
Таргет ↔ Позиция студента в рейтинге: 0.4628
Таргет ↔ Общее количество долгов: 0.4565
Общее количество пересдач ↔ Код направления 3: 4: 0.4229
Контракт ↔ Полных лет на момент поступления: 0.4207
Контракт ↔ Прошло лет с окончания уч. заведения: 0.4016
Контракт ↔ 

In [24]:
subjects_magistr.drop(columns=['БВИ', 'Прошло лет с окончания уч. заведения'], inplace=True)

In [25]:
save_dataset(subjects_bak_spec, "../../data/datasets/subjects_bak_spec.csv")
save_dataset(subjects_magistr, "../../data/datasets/subjects_magistr.csv")

In [26]:
df_numeric = course_1_bak_spec.select_dtypes(include=[np.number])
df_numeric.dropna(inplace=True)
corr_matrix = df_numeric.corr().abs()
np.fill_diagonal(corr_matrix.values, 0)
pairs = (
    corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    .stack()
    .sort_values(ascending=False)
)
top_pairs = pairs.head(20)
for (col1, col2), value in top_pairs.items():
    print(f"{col1} ↔ {col2}: {value:.4f}")

Отдельная квота ↔ Боевые действия: 0.8147
БВИ ↔ олимпиада из перечня, утвержденного МОН РФ (ОШ): 0.7281
Особая квота ↔ Инвалиды: 0.7202
Cумма баллов испытаний ↔ БВИ: 0.7156
Особая квота ↔ Сироты: 0.6477
БВИ ↔ Иностранный абитуриент (МОН): 0.6198
Нуждается в общежитии ↔ fromEkaterinburg: 0.5823
fromEkaterinburg ↔ fromSverdlovskRegion: 0.5711
Иностранный абитуриент (МОН) ↔ Квота для иностранных граждан: 0.5571
Полных лет на момент поступления ↔ Высшее: 0.5368
Cумма баллов испытаний ↔ олимпиада из перечня, утвержденного МОН РФ (ОШ): 0.5211
Полных лет на момент поступления ↔ Заочная: 0.4839
Заочная ↔ СПО: 0.4816
Специалист ↔ Код направления 1: 10: 0.4748
Полных лет на момент поступления ↔ СПО: 0.4557
Cумма баллов испытаний ↔ Иностранный абитуриент (МОН): 0.4435
Код направления 3: 3 ↔ Код направления 3: 4: 0.3883
Нуждается в общежитии ↔ fromSverdlovskRegion: 0.3796
Специалист ↔ Код направления 3: 2: 0.3646
БВИ ↔ Квота для иностранных граждан: 0.3453


In [27]:
df_numeric = course_1_magistr.select_dtypes(include=[np.number])
df_numeric.dropna(inplace=True)
corr_matrix = df_numeric.corr().abs()
np.fill_diagonal(corr_matrix.values, 0)
pairs = (
    corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    .stack()
    .sort_values(ascending=False)
)
top_pairs = pairs.head(20)
for (col1, col2), value in top_pairs.items():
    print(f"{col1} ↔ {col2}: {value:.4f}")

Иностранный абитуриент (МОН) ↔ Квота для иностранных граждан: 0.7431
fromEkaterinburg ↔ fromSverdlovskRegion: 0.6493
Заочная ↔ Код направления 3: 3: 0.5965
Cумма баллов испытаний ↔ Контракт: 0.5250
Иностранный абитуриент (МОН) ↔ Human Development Index: 0.5016
Cумма баллов испытаний ↔ Иностранный абитуриент (МОН): 0.4905
Код направления 1: 27 ↔ Код направления 3: 4: 0.4904
Контракт ↔ Полных лет на момент поступления: 0.4207
Контракт ↔ fromSverdlovskRegion: 0.3809
Cумма баллов испытаний ↔ Квота для иностранных граждан: 0.3645
Human Development Index ↔ Квота для иностранных граждан: 0.3428
Cумма баллов испытаний ↔ Полных лет на момент поступления: 0.3321
Приоритет ↔ Код направления 1: 11: 0.3310
Cумма баллов испытаний ↔ Human Development Index: 0.3209
Cумма баллов испытаний ↔ fromSverdlovskRegion: 0.2923
Код направления 1: 29 ↔ Код направления 3: 3: 0.2831
Контракт ↔ Код направления 1: 11: 0.2759
Приоритет ↔ Контракт: 0.2654
Балл за инд. достижения ↔ Код направления 3: 2: 0.2489
Контракт

In [28]:
df_numeric = subjects_bak_spec.select_dtypes(include=[np.number])
df_numeric.dropna(inplace=True)
corr_matrix = df_numeric.corr().abs()
np.fill_diagonal(corr_matrix.values, 0)
pairs = (
    corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    .stack()
    .sort_values(ascending=False)
)
top_pairs = pairs.head(20)
for (col1, col2), value in top_pairs.items():
    print(f"{col1} ↔ {col2}: {value:.4f}")

Общее количество долгов ↔ Позиция студента в рейтинге: 0.8168
Отдельная квота ↔ Боевые действия: 0.8147
БВИ ↔ олимпиада из перечня, утвержденного МОН РФ (ОШ): 0.7281
Особая квота ↔ Инвалиды: 0.7202
Cумма баллов испытаний ↔ БВИ: 0.7156
Особая квота ↔ Сироты: 0.6477
БВИ ↔ Иностранный абитуриент (МОН): 0.6198
Нуждается в общежитии ↔ fromEkaterinburg: 0.5823
fromEkaterinburg ↔ fromSverdlovskRegion: 0.5711
Иностранный абитуриент (МОН) ↔ Квота для иностранных граждан: 0.5571
Полных лет на момент поступления ↔ Высшее: 0.5368
Общее количество долгов ↔ Код направления 1: 11: 0.5238
Cумма баллов испытаний ↔ олимпиада из перечня, утвержденного МОН РФ (ОШ): 0.5211
Полных лет на момент поступления ↔ Заочная: 0.4839
Таргет ↔ Позиция студента в рейтинге: 0.4826
Заочная ↔ СПО: 0.4816
Специалист ↔ Код направления 1: 10: 0.4748
Полных лет на момент поступления ↔ СПО: 0.4557
Cумма баллов испытаний ↔ Иностранный абитуриент (МОН): 0.4435
Таргет ↔ Общее количество долгов: 0.4302


In [29]:
df_numeric = subjects_magistr.select_dtypes(include=[np.number])
df_numeric.dropna(inplace=True)
corr_matrix = df_numeric.corr().abs()
np.fill_diagonal(corr_matrix.values, 0)
pairs = (
    corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    .stack()
    .sort_values(ascending=False)
)
top_pairs = pairs.head(20)
for (col1, col2), value in top_pairs.items():
    print(f"{col1} ↔ {col2}: {value:.4f}")

Общее количество долгов ↔ Позиция студента в рейтинге: 0.8619
Общее количество пересдач ↔ Код направления 1: 27: 0.8544
Иностранный абитуриент (МОН) ↔ Квота для иностранных граждан: 0.7431
fromEkaterinburg ↔ fromSverdlovskRegion: 0.6493
Заочная ↔ Код направления 3: 3: 0.5965
Cумма баллов испытаний ↔ Контракт: 0.5250
Иностранный абитуриент (МОН) ↔ Human Development Index: 0.5016
Cумма баллов испытаний ↔ Иностранный абитуриент (МОН): 0.4905
Код направления 1: 27 ↔ Код направления 3: 4: 0.4904
Таргет ↔ Позиция студента в рейтинге: 0.4628
Таргет ↔ Общее количество долгов: 0.4565
Общее количество пересдач ↔ Код направления 3: 4: 0.4229
Контракт ↔ Полных лет на момент поступления: 0.4207
Контракт ↔ fromSverdlovskRegion: 0.3809
Cумма баллов испытаний ↔ Квота для иностранных граждан: 0.3645
Human Development Index ↔ Квота для иностранных граждан: 0.3428
Cумма баллов испытаний ↔ Полных лет на момент поступления: 0.3321
Приоритет ↔ Код направления 1: 11: 0.3310
Cумма баллов испытаний ↔ Human Dev